This is coded on an ESP32 for demonstrating neopixels 
Very trivial code, no checking or async work

In [40]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [37]:
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/umqtt.robust/umqtt/robust.py lib/umqtt/
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/umqtt.simple/umqtt/simple.py lib/umqtt/
print(os.listdir("lib/umqtt"))


Sent 204 lines (6411 bytes) to lib/umqtt/simple.py.
['robust.py', 'simple.py']


In [41]:
%sendtofile main.py

import machine, time, network, neopixel

wifiname = "DoESLiverpool-MQTT"
wifipassword = "decafbad00"
mqttbroker = "10.3.141.1"

from machine import Pin, ADC
import time

neo = neopixel.NeoPixel(machine.Pin(12), 74)

time.sleep(0.5)

sta_if = network.WLAN(network.STA_IF)
sta_if.active(True)

neo.fill((10,0,0))
neo.write()

sta_if.connect(wifiname, wifipassword)
while not sta_if.isconnected():
    time.sleep(0.1)
    
neo.fill((0,10,0))
neo.write()
print("connected to wifi")

from umqtt.robust import MQTTClient
client = MQTTClient("neopixelshite", mqttbroker, 1883)
for i in range(100):
    try:
        client.connect()
        break
    except OSError as e:
        print(e)
    gopos(100+800*(i%2))
print("connected to mqtt")
neo.fill((0,0,10))
neo.write()

def sub_cb(topic, msg):
    print((topic, msg))
    try:
        r,g,b = int(msg[:2], 16), int(msg[2:4], 16), int(msg[4:6], 16)
    except ValueError:
        print("ignore")
        return
    neo.fill((r,g,b))
    neo.write()

client.set_callback(sub_cb)
client.subscribe(b"esp32j/fill")

while 1:
    client.check_msg()
    time.sleep_ms(10)


Sent 56 lines (1110 bytes) to main.py.


In [21]:
help(client)



***Connection broken [Input/output error]
You may need to reconnect
Closing serial Serial<id=0x7f21397d0ef0, open=True>(port='/dev/ttyUSB8', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
No serial connected
  %serialconnect to connect
  %esptool to flash the device
  %lsmagic to list commands